# Imports

In [61]:
from pathlib2 import Path
from riotwatcher import LolWatcher, ApiError

import arrow
import json
import pandas as pd

# Functions

In [62]:
def api_key(credentials):
    """Reads in the development API key from a file and checks if it is viable 
    
    Input is a JSON file 

    """
    while True:
        with open(api_key_loc, "r") as json_data:
            credentials = json.load(json_data)
            api_key = credentials["dev_api_key"]
            lol_watcher = LolWatcher(credentials["dev_api_key"])
            try:
                # Use the current API key to check server status
                server_status = lol_watcher.lol_status.shard_data("euw1")
                # break if works
                break
            except ApiError as error:
                # If the current API key does not work input new one
                if error.response.status_code == 403:
                    new_api_key = input("API key is incorrect, enter correct key here.")
                    credentials["dev_api_key"] = new_api_key
                    # Replace the old API key
                    with open(api_key_loc, "w") as json_data:
                        json.dump(credentials, json_data)
    return api_key

In [63]:
def kda(kill, death, assist):
    """Takes the kills, deaths, and assists of a player and calculates the KDA ratio order is kills, deaths, assists 
    
    Input is string

    """
    score = round((kill + assist) / max(1, death), 3)
    return score

In [76]:
def domfac(kill, death, assist):
    """Takes the kills, deaths, and assists of a player and calculates the domination factor order is kills, deaths, assists
    
    Input is string

    """
    score = round(((kill * 2) + assist) - (death * 3), 3)
    return score

In [77]:
print(kda(1,2,3))
print(domfac(1,2,3))

2.0
-1


# Main

In [66]:
# Folder locations

# Project folders
project_folder = Path.cwd().parent

# Raw data storage
data_folder = project_folder / "data"

# Out folder
out_folder = project_folder / "out"

In [67]:
# Set API key
api_key_loc = data_folder / "dev_api_key.json"
lol_watcher = LolWatcher(api_key(api_key_loc)) 

In [68]:
# Set user request parameters

# Set region
region = "euw1"

# Set summonder name 
summoner_name = "alnas" # not case sensitive

### User info

In [69]:
# retrieve info using region and summoner name

user = lol_watcher.summoner.by_name(region, summoner_name)
print(user)

{'id': 'vvnJnOwimLDEuJz55em8ihuAIjfQcfQk0qNTs1xK1SujIEA', 'accountId': 'U8waP8B_8dhhTEMZOZtmyV4vqBfEYHiCNnUMXmPD4EaMeQ', 'puuid': '-0mFh1iO9XEjqTEh8xofNi7wsCx_I7sLHsxhSp7sv8q4sIDuQa_ei3MtfGkud7KyX9PO_ujw76m5yg', 'name': 'Alnas', 'profileIconId': 3382, 'revisionDate': 1610063440000, 'summonerLevel': 103}


In [70]:
# Check rank stats using user id
ranked_stats = lol_watcher.league.by_summoner(region, user["id"])
print(ranked_stats)

[]


### Match info

In [73]:
# set match request parameters

# Set queue
queue = 450

# Set season

In [78]:
# Retrieve last 100 matches (max is 100 matches https://developer.riotgames.com/apis#match-v4) but time range is 1 week 
matches = lol_watcher.match.matchlist_by_account(region=region,
                                                 encrypted_account_id=user["accountId"],
                                                 queue=queue,
                                                 begin_index=0,
                                                 end_index=100)

                                                 
# print(matches.keys())

# Get match details
for i, match in enumerate(matches["matches"][0:1]): # currently last 5 matches
        print(i, match)
        match_detail = lol_watcher.match.by_id(region, match['gameId'])
        
participants = []
for row in match_detail['participants']:
    participants_row = {}
    participants_row['champion'] = row['championId']
    participants_row['spell1'] = row['spell1Id']
    participants_row['spell2'] = row['spell2Id']
    participants_row['win'] = row['stats']['win']
    participants_row['kills'] = row['stats']['kills']
    participants_row['deaths'] = row['stats']['deaths']
    participants_row['assists'] = row['stats']['assists']
    participants_row['KDA'] = kda(row['stats']['kills'], row['stats']['deaths'], row['stats']['assists'])
    participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
    participants_row['goldEarned'] = row['stats']['goldEarned']
    participants_row['champLevel'] = row['stats']['champLevel']
    participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
    participants_row['item0'] = row['stats']['item0']
    participants_row['item1'] = row['stats']['item1']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

0 {'platformId': 'EUW1', 'gameId': 5019622638, 'champion': 62, 'queue': 450, 'season': 13, 'timestamp': 1610062035126, 'role': 'DUO_SUPPORT', 'lane': 'MID'}


,champion,spell1,spell2,win,kills,deaths,assists,KDA,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1
0,38,4,32,True,17,12,23,21,49897,17055,18,19,3100,3157
1,25,4,7,True,4,9,38,19,44646,13643,18,25,6653,3157
2,875,4,32,True,11,14,25,5,65946,15191,18,39,6630,3053
3,51,4,7,True,12,10,20,14,134908,16443,18,98,6672,3006
4,45,4,7,True,12,4,28,40,129098,16076,18,91,6655,3157
5,36,32,4,False,5,9,22,5,40883,12927,18,9,3068,3111
6,236,7,4,False,18,10,18,24,128540,17908,18,121,3111,6672
7,62,4,32,False,8,13,24,1,44348,14544,18,24,3068,3193
8,114,4,32,False,10,13,18,-1,72665,14505,18,53,6630,3111
9,104,14,4,False,7,11,19,0,110942,14616,18,82,6691,6676


### Patch an season info

In [110]:
# example of how to Read time
with open(data_folder / "patches.json") as f:
    data = json.load(f)

patch = data['patches'][-1]  # most recent
utc_timestamp = patch['start']
north_america_timestamp = utc_timestamp + data['shifts']['NA1']
print(north_america_timestamp)
dt = arrow.get(north_america_timestamp)
print(dt.to('US/Pacific'))

1607515200
2020-12-09T04:00:00-08:00


In [ ]:
# example end script for wayyy later

try:
    response = lol_watcher.summoner.by_name(region, summoner_name)
    print(response)
except ApiError as error:
    if error.response.status_code == 429:
        print('Request limit reached retry in {} seconds.'.format(err.headers['Retry-After']))
    elif error.response.status_code == 404: 
        print("Found no match")